# Challenge 5: Multi-Source, Multi-Agent
## Introduction

In this part of the challenge you will add another source of data (structured) to the solution.

### Importing the libraries

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

### Create Environment Variables

**Important:** Make sure you update your `.env` file.

In [2]:
import os, dotenv,sys
dotenv.load_dotenv()

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))


# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")

# Local Folder for the documents
LOCAL_FOLDER = "D:\fsi2023"

# SQL Database
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_DB = os.getenv("SQL_DB")
SQL_USERNAME = os.getenv("SQL_USERNAME")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DRIVER = os.getenv("SQL_DRIVER")

Make Sure you have the ODBC driver installed

In [ ]:
# Check the installed ODBC drivers (you should have installed the "ODBC Driver 18 for SQL Server")
import pyodbc

print(pyodbc.drivers())
assert os.getenv('SQL_DRIVER') in pyodbc.drivers()

## Step 1: Import data to the SQL Database

The database provided is an Azure SQL Database.
Upload the data to the SQL Database using the `./data/fsi/db/create_stock_table.sql` script.

The script will create a table named "stock" in the provided database.
The table contains the following columns:
- **Date** DATE
- **CloseLast** DECIMAL(10, 2) 
- **Volume** INT
- **Open** DECIMAL(10, 2)
- **High** DECIMAL(10, 2)
- **Low** DECIMAL(10, 2)
- **Symbol** VARCHAR(10)

In [4]:
# Create the Connection to the SQL Server Database (sqlalchemy)
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

connection_string = f"Driver={SQL_DRIVER};Server=tcp:{SQL_SERVER},1433;Database={SQL_DB};Uid={SQL_USERNAME};Pwd={SQL_PWD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
quoted_conn_str = quote_plus(connection_string)
# Create the sqlalchemy engine (mssql+pyodbc)
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted_conn_str))


In [ ]:
# Create the Stock Table in the SQL Server Database
with open('../../data/fsi/db/create_stock_table.sql', 'r') as file:
    sql_statements = file.read()

# Execute the SQL Statements
with engine.connect() as connection:
    for command in sql_statements.split('GO'):
        command = command.strip()
        if command:
            connection.execute(text(command))
    connection.execute(text("commit"))

## Step 2: Create a LangChain SQL Agent

In [5]:
# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(api_key = AZURE_OPENAI_API_KEY,  
                      api_version = "2024-06-01",
                      azure_endpoint = AZURE_OPENAI_ENDPOINT,
                      model= AZURE_OPENAI_DEPLOYMENT_NAME,
                      streaming=True)

In [8]:
# Create the LangChain SQL Database Object and the SQL Database Toolkit Object to be used by the agent.


In [9]:
# Create the stock_agent using the Langhcain SQL Agent Class (create_sql_agent)


In [10]:
# Structure the final prompt from the ChatPromptTemplate
# include the system message to the user message
# System message: You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices. If you can't find the answer, say 'I am unable to find the answer.'


In [ ]:
# Test the agent


## Step 3: Create the Multi Agent Solution

In [13]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

## Import the necessary libraries

from typing import Annotated, Sequence
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langgraph.graph import StateGraph, END
from urllib.parse import quote_plus 
from its_a_rag import ingestion
from sqlalchemy import create_engine
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate

In [14]:
# Create the Azure OpenAI Chat Client


In [15]:
# Create the Agent State Class to store the messages between the agents
class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [16]:
# Create the start_agent that analyze the user question and decide if the question is related to stock prices or financial results
def start_agent(state):
    # Import the global llm

    # Prepare the prompt for the agent
    # Prompt Example: You are an agent that needs analyze the user question. \n Question : {input} \n if the question is related to stock prices answer with "stock". \n if the question is related to information about financial results answer with "rag". \n if the question is unclear or you cannot decide answer with "rag". \n only answer with one of the word provided. Your answer (stock/rag):

    # Prepare the chain to be executed

    # invoke the chain

    # take the decision from the response

    # Return the response for the next agent (decision and input required coming fron the Agent State)


In [17]:
## Stock Agent

def stock_agent(state):
    # Import the LLM

    # Create the DB Connection

    # Create the SQL Database Object and the SQL Database Toolkit Object to be used by the agent.

    # Create the agent using the Langhcain SQL Agent Class (create_sql_agent)

    # Structure the final prompt from the ChatPromptTemplate

    # Prepare the response using the invoke method of the agent

    # Return the response for the next agent (output and input required coming fron the Agent State)


In [26]:
## Node rag (this is a clean implementation of the RAG Agent completed in Challenge 4)

def rag_agent(state):
    # Define the LLM

    # Define the index (use the one created in the previous challenge)

    # Define the chain (as it was in the previous challenge)

    # prepare the response using the invoke method of the agent

    # Return the response for the next agent (output and input required coming from the Agent State)


In [27]:
# Create the Agent State Class to store the messages between the agents
# this should include the input, output and decision as strings
class AgentState(TypedDict):
    input: str
    output: str
    decision: str

# Create the 3 steps graph that is going to be working in the bellow "decision" condition
# Add nodes (start_agent, stock_agent, rag_agent) and conditional edges where the decision with be stock or rag
def create_graph():
    # Create the Workflow as StateGraph using the AgentState

    # Add the nodes (start_agent, stock_agent, rag_agent)

    # Add the conditional edge from start -> lamba (decision) -> stock_agent or rag_agent

    # Set the workflow entry point

    # Add the final edges to the END node

    #Compile the workflow


### Step 4: Test the Solution

In [ ]:
## Test Solution

# intantiate the graph (create_graph)
graph = create_graph()

# Use the graph invoke to answer the questions
# Test the graph with various questions
questions = [
    "What are the revenues of Google in the year 2000?",
    "What are the revenues and the operative margins of Alphabet Inc. in 2022 and how it compares with the previous year?",
    "Can you compare and create a table with the revenue of Alphabet Inc., NVIDIA, MICROSOFT, APPLE and AMAZON in years 2023?",
    "Did APPLE repurchase common stock in 2023? create a table of Apple repurchased stock with date, numbers of stocks and values in dollars.",
    "Can you give me the Fiscal Year 2023 Highlights for Apple, Microsoft, Nvidia and Google?"
]

for question in questions:
    result = graph.invoke({"input": question})
    print(result["output"])